In [2]:
%pwd

'/home/gourav/ML/PCB F D temp/research'

In [4]:
import os
os.chdir("../")

In [5]:
%pwd

'/home/gourav/ML/PCB F D temp'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataProcessingself.config:
    source_dir: Path
    raw_data_dir: Path
    processed_data_dir: Path
    split_data_dir: Path

In [7]:
# importing constants and utils
from src.YOLO_V8.constants import *
from src.YOLO_V8.utils.common import read_yaml, create_directories

In [8]:
class self.configurationManager:
    def __init__(
        self,
        self.self.self.self.config_filepath = self.config_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.self.config = read_yaml(self.config_filepath)
        self.params = read_yaml(params_filepath)

    def get_data_process_self.config(self) -> DataProcessingself.config:
        self.config = self.self.config.data_processing 

        data_processing_self.config = DataProcessingself.config(
            source_dir= self.config.source_dir,
            raw_data_dir= self.config.raw_data_dir,
            processed_data_dir= self.config.processed_data_dir,
            split_data_dir = self.config.split_data_dir
        )

        return data_processing_self.config

In [10]:
import pybboxes as pbx

ModuleNotFoundError: No module named 'pybboxes'

In [9]:
import os
import shutil
import pybboxes as pbx
from src.YOLO_V8 import logger
from src.YOLO_V8.utils.common import get_size

ModuleNotFoundError: No module named 'pybboxes'

In [12]:
def path_join(root_path, join_file):
    return os.path.join(root_path, join_file)

In [29]:
class DataProcessing:
    def __init__(self, config: DataProcessingself.config):
        self.self.config = config
        
    
    
    def get_raw_data(self):
        target_dir = self.config.raw_data_dir
        source_dir = self.config.source_dir

        if not os.path.isdir(target_dir):
            os.makedirs(target_dir)
        for groups in os.listdir(source_dir):
            folder_path = os.path.join(source_dir, groups)
            for folder in os.listdir(folder_path):
                target_folder = path_join(target_dir, folder)

                if not os.path.isdir(target_folder):
                    os.makedirs(target_folder)

                for files in os.listdir(folder):
                    files_source_path = os.path.join(folder, files)
                    files_target_path = os.path.join(target_folder, files)
                    shutil.copy(files_source_path, files_target_path)

    def get_processed_images(self, image_folder, image_target_folder):

        for image_file in os.listdir(image_folder):
            image_file_path = path_join(image_folder, image_file)
            target_image_file_path = path_join(image_target_folder, image_file)
            shutil.copy(image_file_path, target_image_file_path)  
    
    def get_coordinate_line(self, line):
        number = line.split(" ")
        get_index = number[-1].split("\n")[0]

        number = list(map(int,number[:-1]))

        # x_center, y_center = (number[2] + number[0]) // 2  , (number[3] + number[1]) // 2
        # box_width, box_height = (number[2] - number[0]) // 2  , (number[3] - number[1]) // 2


        # bbox = list(map(lambda x: x/640, [x_center, y_center, box_width, box_height]))
        W,H = 640,640


        converted_line = pbx.convert_bbox(number, from_type="voc", to_type="yolo", image_size=(W,H))
        converted_line = list(map(str,converted_line))

        converted_line.insert(0,get_index)
        

        result_line = " ".join(converted_line) + "\n"
        
        return result_line
    
    def get_processed_labels(self, labels_folder, labels_target_folder):
        for labels_file in os.listdir(labels_folder):
            labels_file_path = path_join(labels_folder, labels_file)
            target_file_path = path_join(labels_target_folder, labels_file)
            f = open(labels_file_path, "r")
            t = open(target_file_path, "w")
            temp = []
            for lines in f.readlines():
                processed_line = self.get_coordinate_line(lines)
                temp.append(processed_line)
            
            t.writelines(temp)
            f.close()
            t.close()

    
    def processed_data(self):
        source_folder = self.config.raw_data-dir 
        target_folder = self.config.processed_data_dir 

        if not os.path.isdir(target_folder):
            os.makedirs(target_folder)

        for folder in os.listdir(source_folder):

            if folder == "images":
                image_folder = os.path.join(source_folder, folder)
                image_target_folder = os.path.join(target_folder, folder)
                if not os.path.isdir(image_target_folder):
                    os.makedirs(image_target_folder)
                self.get_processed_images(image_folder, image_target_folder)
                

            if folder == "labels":
                labels_folder = os.path.join(source_folder, folder)
                labels_target_folder = os.path.join(target_folder, folder)

                if not os.path.isdir(labels_target_folder):
                    os.makedirs(labels_target_folder)
                
                self.get_processed_labels(labels_folder, labels_target_folder)

In [30]:
self.config = self.configurationManager()
data_processing_self.config = self.config.get_data_process_self.config()
print(data_processing_self.config)

[2023-07-04 21:37:28,496: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-07-04 21:37:28,500: INFO: common: yaml file: params.yaml loaded successfully]
DataProcessingConfig(source_dir='artifacts/data_ingestion/PCBData', raw_data_dir='src/YOLO_V8/data/raw', processed_data_dir='src/YOLO_V8/data/processed', split_data_dir='src/YOLO_V8/data/processed')


In [31]:
data_processing = DataProcessing(data_processing_self.config)


In [32]:
data_processing.processed_data()

In [33]:
import os

In [35]:
p = "/home/gourav/ML/PCB F D temp/src/YOLO_V8/data/processed/labels"
print(len(os.listdir(p)))

1500


In [1]:
import pybboxes as pbx

yolo_normalized = (0.048765432089567184, 0.6583333611488342, 0.09753086417913437, 0.29814815521240234) 

H, W = 640,640

box_voc = pbx.convert_bbox(yolo_normalized, from_type="yolo", to_type="voc", image_size=(W,H))


ModuleNotFoundError: No module named 'pybboxes'

In [2]:
!pip list

Package            Version
------------------ -------
asttokens          2.2.1  
backcall           0.2.0  
comm               0.1.3  
debugpy            1.6.7  
decorator          5.1.1  
ensure             1.0.3  
executing          1.2.0  
importlib-metadata 6.7.0  
ipykernel          6.23.3 
ipython            8.12.2 
jedi               0.18.2 
joblib             1.3.0  
jupyter-client     8.3.0  
jupyter-core       5.3.1  
matplotlib-inline  0.1.6  
nest-asyncio       1.5.6  
numpy              1.24.4 
packaging          23.1   
parso              0.8.3  
pexpect            4.8.0  
pickleshare        0.7.5  
pip                20.0.2 
pkg-resources      0.0.0  
platformdirs       3.8.0  
prompt-toolkit     3.0.38 
psutil             5.9.5  
ptyprocess         0.7.0  
pure-eval          0.2.2  
pybboxes           0.1.6  
Pygments           2.15.1 
python-box         7.0.1  
python-dateutil    2.8.2  
PyYAML             6.0    
pyzmq              25.1.0 
setuptools         44.0.0 
s